<a href="https://colab.research.google.com/github/yulimy0113/LikeLion_AI_bootcamp/blob/master/Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment #1

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
tada_eta = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/data/tada_eta.xlsx')
tada_eta = tada_eta.sample(frac = 1, random_state = 0).reset_index(drop = True)
tada_eta.head()

,ATA,id,api_eta,created_at_kst,month,pickup_lng,pickup_lat,pickup_gu,driver_id,driver_lng,driver_lat,hour
0,6.13,R494SIX8DQBHUE,8.32,2019-12-26T22:16:29,12,126.913779,37.549010,마포구,DTS51017,126.913082,37.527922,22
1,5.37,R3CB3T7VXNJCA2,3.05,2019-12-11T11:01:18,12,127.089688,37.501710,송파구,DMS89350,127.084503,37.498104,11
2,6.27,R3UFI26UQ6ZWIS,5.00,2019-07-25T19:53:40,7,126.972378,37.569360,종로구,DWZ29749,126.972541,37.576297,19
3,10.52,R2FLJESR1EWVMT,8.47,2019-12-21T21:57:55,12,126.974081,37.519973,용산구,DPT41134,127.001186,37.526196,21
4,12.45,R28S4XRQC2ZEB,6.65,2019-07-07T12:19:43,7,127.025221,37.524640,강남구,DPK84123,127.026035,37.512465,12


사용하지 않을 feature 제거 : id, created_at_kst, month, driver_id

In [6]:
tada_eta = tada_eta.drop(['id', 'created_at_kst', 'month', 'driver_id'], 1)

위도/경도 데이터를 유클리디안거리로 계산한 column 추가

In [7]:
tada_eta['distance'] = ((tada_eta['pickup_lat']-tada_eta['driver_lat'])**2 + (tada_eta['pickup_lng']-tada_eta['driver_lng'])**2)*10000
tada_eta.head()

,ATA,api_eta,pickup_lng,pickup_lat,pickup_gu,driver_lng,driver_lat,hour,distance
0,6.13,8.32,126.913779,37.549010,마포구,126.913082,37.527922,22,4.452198
1,5.37,3.05,127.089688,37.501710,송파구,127.084503,37.498104,11,0.398907
2,6.27,5.00,126.972378,37.569360,종로구,126.972541,37.576297,19,0.481470
3,10.52,8.47,126.974081,37.519973,용산구,127.001186,37.526196,21,7.734117
4,12.45,6.65,127.025221,37.524640,강남구,127.026035,37.512465,12,1.488885


Ordinal encoding : 구 데이터를 범주형으로 변환

In [8]:
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder(dtype = np.int32)
ordinal = enc.fit_transform(np.asarray(tada_eta['pickup_gu']).reshape(-1,1))

In [9]:
tada_eta['pickup_gu'] = ordinal[:, 0]
tada_eta.head()

,ATA,api_eta,pickup_lng,pickup_lat,pickup_gu,driver_lng,driver_lat,hour,distance
0,6.13,8.32,126.913779,37.549010,12,126.913082,37.527922,22,4.452198
1,5.37,3.05,127.089688,37.501710,17,127.084503,37.498104,11,0.398907
2,6.27,5.00,126.972378,37.569360,22,126.972541,37.576297,19,0.481470
3,10.52,8.47,126.974081,37.519973,20,127.001186,37.526196,21,7.734117
4,12.45,6.65,127.025221,37.524640,0,127.026035,37.512465,12,1.488885


### Model#1
 * Dimentionality reduction > PCA

In [14]:
data_pca = pd.concat([tada_eta.iloc[:, 2:4], tada_eta.iloc[:, 5:7]], axis = 1)
data_pca.head()

,pickup_lng,pickup_lat,driver_lng,driver_lat
0,126.913779,37.549010,126.913082,37.527922
1,127.089688,37.501710,127.084503,37.498104
2,126.972378,37.569360,126.972541,37.576297
3,126.974081,37.519973,127.001186,37.526196
4,127.025221,37.524640,127.026035,37.512465


In [16]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA

In [32]:
data_pca = StandardScaler().fit_transform(data_pca)
data_pca = PCA(n_components = 2).fit_transform(data_pca)
data1 = tada_eta.drop('pickup_gu', 1)
data1['pca_1'] = data_pca[:, 0]
data1['pca_2'] = data_pca[:, 1]

In [33]:
data1.head()

,ATA,api_eta,pickup_lng,pickup_lat,driver_lng,driver_lat,hour,distance,pca_1,pca_2
0,6.13,8.32,126.913779,37.549010,126.913082,37.527922,22,4.452198,-0.016054,1.406748
1,5.37,3.05,127.089688,37.501710,127.084503,37.498104,11,0.398907,0.003563,-1.480490
2,6.27,5.00,126.972378,37.569360,126.972541,37.576297,19,0.481470,-0.079022,1.174282
3,10.52,8.47,126.974081,37.519973,127.001186,37.526196,21,7.734117,-0.011418,0.157793
4,12.45,6.65,127.025221,37.524640,127.026035,37.512465,12,1.488885,-0.011290,-0.309461


In [34]:
X_tn_1 = data1.loc[:12000, 'api_eta': ]
X_te_1 = data1.loc[12000:, 'api_eta': ]
y_tn_1 = data1.loc[:12000, 'ATA']
y_te_1 = data1.loc[12000:, 'ATA']

In [37]:
params_1 = {
    'max_iter' : [500, 1000, 1500],
    'max_depth' : [2, 3, 4],
    'min_samples_leaf' : [10, 15, 20],
    'learning_rate' : [0.001, 0.005, 0.01, 0.05]
}

In [38]:
grid_hgbr_1 = GridSearchCV(estimator=HistGradientBoostingRegressor(), param_grid = params_1,
                         cv = 3, n_jobs = -1,
                         scoring = 'neg_mean_squared_error', verbose = 2)
grid_hgbr_1.fit(X_tn_1, y_tn_1)
params_fit_1 = grid_hgbr_1.best_params_
print(params_fit_1)

Fitting 3 folds for each of 108 candidates, totalling 324 fits
{'learning_rate': 0.01, 'max_depth': 2, 'max_iter': 1500, 'min_samples_leaf': 20}


In [39]:
model_1 = HistGradientBoostingRegressor(**params_fit_1).fit(X_tn_1, y_tn_1)

print("Errors of Regression model #1")
mse_pca = mean_squared_error(y_te_1, model_1.predict(X_te_1))
print("The mean squared error(MSE) on test set: {:.4f}".format(mse_pca))
print("The initial error of API ETA on test set: {:.4f}".format(mean_squared_error(y_te_1, X_te_1.loc[:,'api_eta']) ))

mae_pca = mean_absolute_error(y_te_1, model_1.predict(X_te_1))
print("The mean absolute error (MAE) on test set: {:.4f}".format(mae_pca))
print("The initial error of API ETA on test set: {:.4f}".format(mean_absolute_error(y_te_1, X_te_1.loc[:,'api_eta']) ))

Errors of Regression model #1
The mean squared error(MSE) on test set: 8.3026
The initial error of API ETA on test set: 13.7135
The mean absolute error (MAE) on test set: 2.0755
The initial error of API ETA on test set: 2.7047


### Model #2
 * Clustering > DBSCAN

In [58]:
from sklearn.cluster import DBSCAN

In [57]:
data2 = tada_eta

In [59]:
std_data = pd.concat([tada_eta.iloc[:, 2:4], tada_eta.iloc[:, 5:7]], axis = 1)
std_data = StandardScaler().fit_transform(std_data)

In [60]:
data2 = data2.drop(['pickup_lng', 'pickup_lat', 'driver_lng', 'driver_lat'], 1)
data2 = pd.concat([data2, pd.DataFrame(std_data)], axis = 1)

In [61]:
data2.head()

,ATA,api_eta,pickup_gu,hour,distance,pca_1,pca_2,0,1,2,3
0,6.13,8.32,12,22,4.452198,-0.022701,1.579482,-1.543183,0.690330,-0.018221,0.019437
1,5.37,3.05,17,11,0.398907,0.005038,-1.662280,1.520313,-0.828541,0.052978,-0.022447
2,6.27,5.00,22,19,0.481470,-0.111744,1.318472,-0.522667,1.343788,0.006475,0.087388
3,10.52,8.47,20,21,7.734117,-0.016146,0.177168,-0.493010,-0.242103,0.018373,0.017013
4,12.45,6.65,0,12,1.488885,-0.015965,-0.347460,0.397602,-0.092230,0.028694,-0.002274


In [71]:
dbscan = DBSCAN(eps = 3, min_samples = 10).fit_predict(data2)
data2.loc[:, 'dbscan'] = dbscan
data2.loc[:, 'dbscan'].value_counts()

 0     4319
 2     4158
 1     3655
-1     1511
 3      107
 4       52
 5       37
 8       24
 9       14
 6       14
 7       14
 10      11
Name: dbscan, dtype: int64

In [72]:
data2.head()

,ATA,api_eta,pickup_gu,hour,distance,pca_1,pca_2,0,1,2,3,dbscan
0,6.13,8.32,12,22,4.452198,-0.022701,1.579482,-1.543183,0.690330,-0.018221,0.019437,0
1,5.37,3.05,17,11,0.398907,0.005038,-1.662280,1.520313,-0.828541,0.052978,-0.022447,0
2,6.27,5.00,22,19,0.481470,-0.111744,1.318472,-0.522667,1.343788,0.006475,0.087388,1
3,10.52,8.47,20,21,7.734117,-0.016146,0.177168,-0.493010,-0.242103,0.018373,0.017013,1
4,12.45,6.65,0,12,1.488885,-0.015965,-0.347460,0.397602,-0.092230,0.028694,-0.002274,2


In [73]:
X_tn_2 = data2.loc[:12000, 'api_eta':]
X_te_2 = data2.loc[12000:, 'api_eta':]
y_tn_2 = data2.loc[:12000, 'ATA']
y_te_2 = data2.loc[12000:, 'ATA']

In [75]:
X_tn_2.head()

,api_eta,pickup_gu,hour,distance,pca_1,pca_2,0,1,2,3,dbscan
0,8.32,12,22,4.452198,-0.022701,1.579482,-1.543183,0.690330,-0.018221,0.019437,0
1,3.05,17,11,0.398907,0.005038,-1.662280,1.520313,-0.828541,0.052978,-0.022447,0
2,5.00,22,19,0.481470,-0.111744,1.318472,-0.522667,1.343788,0.006475,0.087388,1
3,8.47,20,21,7.734117,-0.016146,0.177168,-0.493010,-0.242103,0.018373,0.017013,1
4,6.65,0,12,1.488885,-0.015965,-0.347460,0.397602,-0.092230,0.028694,-0.002274,2


In [76]:
params_2 = {
    'max_iter' : [500, 1000, 1500],
    'max_depth' : [2, 3, 4, 5],
    'min_samples_leaf' : [10, 15, 20],
    'learning_rate' : [0.001, 0.005, 0.01, 0.05],
    'categorical_features' : [1]
}

In [77]:
grid_hgbr_2 = GridSearchCV(estimator=HistGradientBoostingRegressor(), param_grid = params_2,
                         cv = 3, n_jobs = -1,
                         scoring = 'neg_mean_squared_error', verbose = 2)
grid_hgbr_2.fit(X_tn_2, y_tn_2)
params_fit_2 = grid_hgbr_2.best_params_
print(params_fit_2)

Fitting 3 folds for each of 144 candidates, totalling 432 fits
{'categorical_features': 1, 'learning_rate': 0.05, 'max_depth': 3, 'max_iter': 500, 'min_samples_leaf': 15}


In [78]:
model_2 = HistGradientBoostingRegressor(**params_fit_2).fit(X_tn_2, y_tn_2)

print("Errors of Regression model #1")
mse_db = mean_squared_error(y_te_2, model_2.predict(X_te_2))
print("The mean squared error(MSE) on test set: {:.4f}".format(mse_db))
print("The initial error of API ETA on test set: {:.4f}".format(mean_squared_error(y_te_2, X_te_2.loc[:,'api_eta']) ))

mae_db = mean_absolute_error(y_te_2, model_2.predict(X_te_2))
print("The mean absolute error (MAE) on test set: {:.4f}".format(mae_db))
print("The initial error of API ETA on test set: {:.4f}".format(mean_absolute_error(y_te_2, X_te_2.loc[:,'api_eta']) ))

Errors of Regression model #1
The mean squared error(MSE) on test set: 6.7584
The initial error of API ETA on test set: 13.7135
The mean absolute error (MAE) on test set: 1.9397
The initial error of API ETA on test set: 2.7047
